In [1]:
import pandas as pd
import numpy as np

# Outputs


In [2]:
notifs = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Notificaciones TB/GTM - TB all notifications 2012-Sep2018.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7,8,9,11,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,69,70,72,74,78,79,80,81,83,84,85,87,89,90,91,93,94,98,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
notifs["Year"] = np.floor(notifs.YearMonth/100)

In [4]:
casesNotif = notifs[(notifs.Year<2018) & notifs.COD_MUNI.notna()].assign(municipality = lambda x: x.COD_MUNI).assign(date=lambda x: x.Year.astype(int))\
.assign(department = lambda x: (np.floor(x.COD_MUNI/100)).astype(int) ).groupby(["date", "department", "municipality"]).apply(len)

In [5]:
casesNotif = casesNotif[casesNotif.index.get_level_values(1)!=0].rename("Cases Notified_value")

In [6]:
extraM2017 = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Extramuros2017.csv", index_col=0)

In [7]:
acfcases = extraM2017[extraM2017.Year.notna()].assign(municipality = lambda x: x.MuniCode.astype(int)).assign(date = lambda x: x.Year.astype(int)).\
assign(department = lambda x: x.DeptoCode.astype(int)).groupby(["date", "department", "municipality"])["bacteriologicos.extramuros"].sum()
acfcases = acfcases.rename("Additional Cases Detected via ACF_value")

In [8]:
#acfcases = acfcases.reindex(casesNotif.index)

In [9]:
# Cases Started on Treatment_value
# MDR Cases Started Treatment_value

cohortes = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/COHORTES/GTM - Tx cohort data 2012-2016.csv")

In [10]:
cohortes.head()

,Unnamed: 0,col_name,deptocode,row_name,row_name_B,table,value,year
0,0,I TRIMESTRE,0,CURADOS,COMPLETED TREATMENT,Nuevos Pulmonares Genexpert,6.0,2016
1,1,II TRIMESTRE,0,CURADOS,COMPLETED TREATMENT,Nuevos Pulmonares Genexpert,7.0,2016
2,2,III TRIMESTRE,0,CURADOS,COMPLETED TREATMENT,Nuevos Pulmonares Genexpert,4.0,2016
3,3,IV TRIMESTRE,0,CURADOS,COMPLETED TREATMENT,Nuevos Pulmonares Genexpert,8.0,2016
4,4,TOTAL,0,CURADOS,COMPLETED TREATMENT,Nuevos Pulmonares Genexpert,25.0,2016


In [11]:
pivot_cohortes = cohortes[(cohortes.col_name.isin(["TOTAL"])) & 
       (cohortes.table.isin(["Nuevos Pulmonares BK+", "Nuevos Pulmonares BK-", "Nuevos Pediatricos", 
                          "Nuevos Extrapulmonares", "Nuevos TB/VIH", "Retratamiento"])) &
       (cohortes.deptocode!=0)].\
    groupby(["year", "deptocode", "row_name_B"]).value.sum().unstack(2)

In [12]:
pivot_cohortes.head()

row_name_B      COMPLETED TREATMENT  DEATHS  LOST TO FOLLOW-UP  REFERRED
year deptocode                                                          
2012 1                        594.0    28.0               23.0       3.0
     2                         21.0     4.0                1.0       0.0
     3                         39.0     0.0                0.0       0.0
     4                         31.0     4.0                0.0       1.0
     5                        508.0    43.0               43.0       9.0

In [13]:
initiatedTx = pivot_cohortes.sum(axis=1)

In [14]:
initiatedTx = initiatedTx.rename_axis(index = { "year": "date", "deptocode": "department" }).rename("Cases Started on Treatment_value")

In [15]:
initiatedTx2 = initiatedTx.reindex(casesNotif.index)

In [16]:
# Reindex is not bad in this case??
initiatedTx.sum(), initiatedTx2.groupby([initiatedTx2.index.get_level_values(0), initiatedTx2.index.get_level_values(1)]).first().sum()

(17116.0, 17116.0)

In [17]:
tbquimio = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Notificaciones TB/GTM - TB quimio 2012-Sep2018.csv",
                     index_col = 0, low_memory = False)

In [18]:
tbquimio.head()

,NOMBRES,DIRECCION,MUNICIPIO,DEPARTAMENTO,SERVICIODESALUD,SEXO,EDAD,RANGOEDAD,PESOLBS,PESOKG,...,CONTROL7_GX,OBSERVACIONES,PRIORIZACION,PESOLB,NotificationDate,YearMonth_,DEPTO_CORRECTED,i,COD_DEPT,COD_MUNI
1,NaN,NaN,GRANADOS,BAJA VERAPAZ,"CS, GRANADOS",f,1.0,000 - 004,13.4,6.0909090909090908,...,NaN,NaN,NaN,NaN,NaN,NaN,BAJA VERAPAZ,165,15,1505
2,NaN,NaN,SALAMA,BAJA VERAPAZ,"PS, LLANO GRANDE",f,8.0,005 - 009,50.8,23.090909090909086,...,NaN,NaN,NaN,NaN,NaN,NaN,BAJA VERAPAZ,167,15,1501
3,NaN,NaN,SALAMA,BAJA VERAPAZ,"PS, LLANO GRANDE",m,5.0,005 - 009,35,15.909090909090908,...,NaN,NaN,NaN,NaN,NaN,NaN,BAJA VERAPAZ,168,15,1501
4,NaN,NaN,SALAMA,BAJA VERAPAZ,"PS, LLANO GRANDE",m,6.0,005 - 009,36,16.363636363636363,...,NaN,NaN,NaN,NaN,NaN,NaN,BAJA VERAPAZ,169,15,1501
5,NaN,NaN,SALAMA,BAJA VERAPAZ,"PS, LLANO GRANDE",m,2.0,000 - 004,24.8,11.272727272727272,...,NaN,NaN,NaN,NaN,NaN,NaN,BAJA VERAPAZ,170,15,1501


In [19]:
tbquimio["Year"] = np.floor(tbquimio.YearMonth/100)

In [20]:
plhivipt= tbquimio[tbquimio.VIH=="Reactive"].assign(department = lambda x: np.floor(x.COD_MUNI/100))\
.groupby(["Year", "department", "COD_MUNI"]).apply(len)

In [21]:
plhivipt = plhivipt.rename("PLHIV started on IPT_value")\
.rename_axis(index={"Year": "date", "COD_MUNI": "municipality"}) #.reindex(casesNotif.index)

In [22]:
# This is only contacts in notifications database. I am not sure if its ok.

In [23]:
childipt= tbquimio[(tbquimio.Year<2018) & (tbquimio.EDAD <= 5)].assign(department = lambda x: np.floor(x.COD_MUNI/100))\
.groupby(["Year", "department", "COD_MUNI"]).apply(len).rename("Children in Contact with TB Started IPT_value")

In [24]:
childipt = childipt.rename_axis(index={"Year": "date", "COD_MUNI": "municipality"}) #.reindex(casesNotif.index)

In [25]:
# Cases Notified in Prisons_value
pplcases = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/PPL-conteos-2014-2017.csv", index_col=0)

In [26]:
pplcases = pplcases.assign(department = lambda x : np.floor(x.municipio/100)).rename(columns=\
{"n": "Cases Notified in Prisons_value", "YEAR": "date", "municipio": "municipality"}).set_index(["date", "department", "municipality"])

In [27]:
pplcases = pplcases.reindex(casesNotif.index)

In [37]:
ppldata = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/tb-ppl-2014-2017.csv")

In [48]:
ppldata["TXDATE"] = pd.to_datetime(ppldata.TXDATE, errors="coerce")

In [49]:
# Sólo una persona no recibió tratamiento.
ppldata.TXDATE.isna().sum()

1

In [28]:
outputs = pd.merge(casesNotif, acfcases, left_index=True, right_index=True, how="outer")\
            .merge(initiatedTx, left_index=True, right_index=True, how="outer")\
            .merge(plhivipt, left_index=True, right_index=True, how="outer")\
            .merge(childipt, left_index=True, right_index=True, how="outer")\
            .merge(pplcases, left_index=True, right_index=True, how="outer")\

In [29]:
outputs

Cases Notified_value  \
date   department municipality                         
2012.0 1.0        101.0                        318.0   
                  102.0                          5.0   
                  103.0                         15.0   
                  104.0                          1.0   
                  105.0                          4.0   
                  106.0                         30.0   
                  107.0                         20.0   
                  108.0                        128.0   
                  109.0                          5.0   
                  110.0                         54.0   
                  111.0                          6.0   
                  113.0                          9.0   
                  114.0                         35.0   
                  115.0                        102.0   
                  116.0                         31.0   
                  117.0                         30.0   
       2.0        201.0                          2.0   
                  203.0                         11.0   
                  205.0                          4.0   
                  206.0                          3.0   
                  207.0                          4.0   
       3.0        301.0                         10.0   
                  304.0                          5.0   
                  305.0                          2.0   
                  306.0                          8.0   
                  310.0                          2.0   
                  311.0                          5.0   
                  312.0                          6.0   
                  313.0                          3.0   
                  314.0                          4.0   
...                                              ...   
2017.0 19.0       1904.0                         7.0   
                  1905.0                         1.0   
                  1906.0                         3.0   
                  1907.0                         2.0   
                  1908.0                         1.0   
                  1909.0                         1.0   
                  1910.0                         1.0   
                  1911.0                         7.0   
       20.0       2001.0                        27.0   
                  2002.0                         4.0   
                  2004.0                         8.0   
                  2005.0                        16.0   
                  2007.0                         1.0   
                  2010.0                         1.0   
                  2011.0                        10.0   
       21.0       2101.0                        17.0   
                  2102.0                         1.0   
                  2103.0                         3.0   
                  2107.0                         2.0   
       22.0       2201.0                        19.0   
                  2202.0                         1.0   
                  2204.0                         3.0   
                  2205.0                         8.0   
                  2206.0                         2.0   
                  2207.0                         3.0   
                  2211.0                        10.0   
                  2212.0                         5.0   
                  2213.0                         1.0   
                  2214.0                         4.0   
                  2217.0                         3.0   

                                Additional Cases Detected via ACF_value  \
date   department municipality                                            
2012.0 1.0        101.0                                             NaN   
                  102.0                                             NaN   
                  103.0                                             NaN   
                  104.0                                             NaN   
                  105.0                              